# Speed, Memory, and Disk Comparisons

In this notebook, we'll offer some rough comparisons of the computational performance implications of ESGPT vs. other competing pipelines. We'll focus these comparisons on several metrics:
  1. The time, runtime memory, and final disk space required to construct, pre-process, and store an ESGPT dataset relative to other pipelines, where applicable.
  2. The initialization time, iteration speed, and GPU memory costs for producing batches of data within the ESGPT framework vs. other systems.
  
In particular, we'll compare (or justify why they are inappropriate comparators) against the following pipelines:
  1. TemporAI
  2. OMOP-Learn
  3. FIDDLE
  4. MIMIC-Extract
  
We'll make these comparisons leveraging the synthetic data distributed with ESGPT's sample tutorial, but this code can also be ported to any other dataset to run these profiles locally.

In [1]:
%load_ext memory_profiler

import os
import rootutils
import sys

root = rootutils.setup_root(os.path.abspath(""), dotenv=True, pythonpath=True, cwd=False)
sys.path.append(os.environ["EVENT_STREAM_PATH"])

In [2]:
import os
import numpy as np
import torch

from collections import defaultdict
from datetime import datetime, timedelta
from humanize import naturalsize, naturaldelta
from pathlib import Path
from sparklines import sparklines
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
from typing import Callable

from EventStream.data.dataset_polars import Dataset
from EventStream.data.config import PytorchDatasetConfig
from EventStream.data.types import PytorchBatch
from EventStream.data.pytorch_dataset import PytorchDataset

In [3]:
COHORT_NAME = "MIMIC_IV/ESD_07-23-23_150GB_10cpu-1"
PROJECT_DIR = Path(os.environ["PROJECT_DIR"])
dataset_dir = PROJECT_DIR / "data" / COHORT_NAME

First, let's check and see how much disk space the dataset uses, and in what components

In [4]:
total_dataset_size = sum(f.stat().st_size for f in dataset_dir.glob('**/*') if f.is_file())
DL_reps_size = sum(f.stat().st_size for f in (dataset_dir / "DL_reps").glob('**/*') if f.is_file())
just_dataset_size = total_dataset_size - DL_reps_size

if (dataset_dir / "flat_reps").is_dir():
    flat_reps_size = sum(f.stat().st_size for f in (dataset_dir / "flat_reps").glob('**/*') if f.is_file())
    just_dataset_size -= flat_reps_size
    flat_reps_lines = [f"  * {naturalsize(flat_reps_size)} for the flat representation dataframes."]
else:
    flat_reps_lines = []

lines = [
    f"The total dataset takes up {naturalsize(total_dataset_size)} on disk, which includes:",
    f"  * {naturalsize(just_dataset_size)} for the core dataset.",
    f"  * {naturalsize(DL_reps_size)} for the deep-learning representation dataframes.",
] + flat_reps_lines

print('\n'.join(lines))

The total dataset takes up 65.5 GB on disk, which includes:
  * 1.0 GB for the core dataset.
  * 2.4 GB for the deep-learning representation dataframes.
  * 62.1 GB for the flat representation dataframes.


First, we'll note that loading a dataset doesn't require much of either resource. This is because the data is loaded lazily, so complex dataframe elements aren't loaded until they are needed. 

In [14]:
%%time
%%memit

ESD = Dataset.load(dataset_dir)

peak memory: 866.52 MiB, increment: 3.37 MiB
CPU times: user 409 ms, sys: 37.3 ms, total: 446 ms
Wall time: 1.03 s


In [7]:
%%time
%%memit

s_df = ESD.subjects_df
e_df = ESD.events_df
dm_df = ESD.dynamic_measurements_df

Loading subjects from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_07-23-23_150GB_10cpu-1/subjects_df.parquet...
Loading events from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_07-23-23_150GB_10cpu-1/events_df.parquet...
Loading dynamic_measurements from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_07-23-23_150GB_10cpu-1/dynamic_measurements_df.parquet...
peak memory: 18050.05 MiB, increment: 17634.09 MiB
CPU times: user 26.4 s, sys: 13.9 s, total: 40.2 s
Wall time: 11 s


## Pytorch Dataset Stats
Now let's load a pytorch dataset and examine iteration speed and GPU memory cost:

In [4]:
def summarize(arr: list[float], strify: Callable[float, str] = naturalsize) -> str:
    mean, std, mn, mx = np.mean(arr), np.std(arr), np.min(arr), np.max(arr)
    simple_summ = f"{strify(mean)} ± {strify(std)} ({strify(mn)}-{strify(mx)})"
    
    if len(arr) < 25: return simple_summ
    
    hist_vals, hist_bins = np.histogram(arr)
    lines = [simple_summ, "Histogram:"]
    sparkline = sparklines(hist_vals)
    
    lines.extend(sparkline)
    left_end = strify(hist_bins[0])
    right_end = strify(hist_bins[1])
    W = len(sparkline[0]) - len(left_end) - len(right_end)
    
    if W > 0:
        lines.append(f"{left_end}{'-'*W}{right_end}")
    else:
        lines.append(f"o {left_end} (left endpoint)")
        lines.append(f"{'-'*(len(sparkline[0])-1)}o {right_end} (right endpoint)")
    return '\n'.join(lines)

def summarize_times(arr: list[float, timedelta]):
    as_seconds = [x / timedelta(seconds=1) for x in arr]
    return summarize(as_seconds, strify=lambda x: str(timedelta(seconds=x)))

In [5]:
def profile_batch_iteration_speed_and_cost(
    batch_size: int,
    pyd: Dataset,
    n_iter_samples: int = 30,
    collate_fn: Callable | None = None,
):
    def make_dataloader():
        if collate_fn is None:
            return DataLoader(pyd, batch_size=batch_size, shuffle=True)
        return DataLoader(pyd, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)

    dataloader = make_dataloader()
    batch_sizes = defaultdict(list)
    total_sizes = []
    for batch in tqdm(dataloader, leave=False):
        total_size = 0
        for k, v in batch.items():
            if v is None: continue
            el_size = v.element_size() * v.nelement()
            batch_sizes[k].append(el_size)
            total_size += el_size
        total_sizes.append(total_size)

    batch_iteration_times = []
    for samp in tqdm(list(range(n_iter_samples)), leave=False, desc="Sampling Dataloader Iteration Speed"):
        dataloader = make_dataloader()
        st = datetime.now()
        for batch in tqdm(dataloader, leave=False, desc="Sampling Batch"):
            pass
        batch_iteration_times.append((datetime.now() - st) / len(dataloader))

    print(
        f"Iterating through an entire dataloader of {len(dataloader)} batches of size {batch_size} "
        f"took the following time per batch:\n{summarize_times(batch_iteration_times)}\n\n"
        f"Total batch size:\n{summarize(total_sizes)}"
    )
    for k, v in batch_sizes.items():
        print(f"  Size of {k}:\n    {summarize(v)}")

In [10]:
%%time
%%memit
pyd_config = PytorchDatasetConfig(
    save_dir=ESD.config.save_dir,
    max_seq_len=1024,
)
pyd = PytorchDataset(config=pyd_config, split='train')

peak memory: 19293.29 MiB, increment: 7612.77 MiB
CPU times: user 57.4 s, sys: 7.93 s, total: 1min 5s
Wall time: 42.5 s


In [11]:
%%time
%%memit

profile_batch_iteration_speed_and_cost(batch_size=16, pyd=pyd, n_iter_samples=5, collate_fn=pyd.collate)

  0%|          | 0/607 [00:00<?, ?it/s]

Sampling Dataloader Iteration Speed:   0%|          | 0/5 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/607 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/607 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/607 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/607 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/607 [00:00<?, ?it/s]

Iterating through an entire dataloader of 607 batches of size 16 took the following time per batch:
0:00:00.416088 ± 0:00:00.004160 (0:00:00.410064-0:00:00.422636)

Total batch size:
67.2 MB ± 35.0 MB (18.3 MB-410.6 MB)
Histogram:
██▃▁▁▁▁▁▁▁
o 18.3 MB (left endpoint)
---------o 57.6 MB (right endpoint)
  Size of event_mask:
    10.9 kB ± 3.6 kB (3.3 kB-16.4 kB)
Histogram:
▁▃▄▅▅▅▄▃▃█
o 3.3 kB (left endpoint)
---------o 4.6 kB (right endpoint)
  Size of time_delta:
    43.8 kB ± 14.4 kB (13.1 kB-65.5 kB)
Histogram:
▁▃▄▅▅▅▄▃▃█
o 13.1 kB (left endpoint)
---------o 18.3 kB (right endpoint)
  Size of static_indices:
    127 Bytes ± 3 Bytes (48 Bytes-128 Bytes)
Histogram:
▁▁▁▁▁▁▁▁▁█
o 48 Bytes (left endpoint)
---------o 56 Bytes (right endpoint)
  Size of static_measurement_indices:
    127 Bytes ± 3 Bytes (48 Bytes-128 Bytes)
Histogram:
▁▁▁▁▁▁▁▁▁█
o 48 Bytes (left endpoint)
---------o 56 Bytes (right endpoint)
  Size of dynamic_indices:
    25.6 MB ± 13.3 MB (7.0 MB-156.4 MB)
Histogram:
██▃▁

## Other Pipelines
### TemporAI Format

In [6]:
import pandas as pd
import polars as pl
import polars.selectors as cs

In [6]:
def ESD_to_temporai(ESD: Dataset, n_subjects: int | None = 1000) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Converts an ESD data format into a TemporAI dataset format."""
    
    if n_subjects is None:
        subject_ids = list(ESD.split_subjects['train'])
    else:
        subject_ids = list(np.random.default_rng(1).choice(list(ESD.split_subjects['train']), size=n_subjects))

    static_df = (
        ESD.subjects_df
        .filter(pl.col('subject_id').is_in(subject_ids))
        .select(
            'subject_id',
            *[pl.col(c) for c, cfg in ESD.measurement_configs.items() if cfg.temporality == 'static']
        )
        .to_pandas()
        .set_index("subject_id")
    )
    
    # For the time-series dataframe, as they need only one row per subject ID, timestamp, we need to use the wide
    # format of the flat representation. 
    
    flat_reps_dir = ESD.config.save_dir / "flat_reps" / "raw"
    if not flat_reps_dir.is_dir():
        raise FileNotFoundError(f"Must have pre-cached flat representations at {flat_reps_dir}!")
        
    ts_dfs = []
    for fp in tqdm(list(flat_reps_dir.glob("*/*.parquet")), leave=False, desc="Flat Rep Files"):
        new_df = (
            pl.scan_parquet(fp)
            .filter(pl.col('subject_id').is_in(subject_ids))
            .select("subject_id", "timestamp", cs.starts_with("dynamic"))
            .collect(streaming=True)
            .to_pandas()
            .set_index(["subject_id", "timestamp"])
        )
        
        if len(new_df) > 0:
            ts_dfs.append(new_df)
    
    return static_df, ts_dfs

In [14]:
%%time
%%memit
# We need to convert to a flat format prior to getting temporai representations.
# The performance #s here are not reliable as these files may be already generated.
ESD.cache_flat_representation(
    subjects_per_output_file=None,
    feature_inclusion_frequency=None,
    do_overwrite=False,
    do_update=True,
)

Standardizing chunk size to existing record (250).


Flattening Splits:   0%|          | 0/3 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/38 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/4 [00:00<?, ?it/s]

Subject chunks:   0%|          | 0/4 [00:00<?, ?it/s]

peak memory: 17883.32 MiB, increment: 1.73 MiB
CPU times: user 4.47 s, sys: 4.62 s, total: 9.09 s
Wall time: 9.51 s


In [7]:
%%time
%%memit

# This is for subsampling to make the full dataset less expensive to load.
# Even subsampling down to 1000 patients makes the temporai fully padded files
# too expensive to store in 75GB of memory.
n_subjects = 250
temporai_subsample_rate = n_subjects / len(ESD.split_subjects['train'])
temporai_static, temporai_ts_L = ESD_to_temporai(ESD, n_subjects=n_subjects)

Loading subjects from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_07-23-23_150GB_10cpu-1/subjects_df.parquet...


Flat Rep Files:   0%|          | 0/46 [00:00<?, ?it/s]

peak memory: 7941.90 MiB, increment: 7538.58 MiB
CPU times: user 5min 45s, sys: 1min 44s, total: 7min 30s
Wall time: 2min 57s


In [8]:
%%time
%%memit
temporai_ts = pd.concat(temporai_ts_L, axis=0)

peak memory: 10914.00 MiB, increment: 4157.78 MiB
CPU times: user 7.63 s, sys: 3.33 s, total: 11 s
Wall time: 11.5 s


In [9]:
print(
    f"TemporAI uses two dataframes, a static dataframe of shape {temporai_static.shape} "
    f"and a time series dataframe of shape {temporai_ts.shape}. "
    f"This is sub-sampled at a rate of {(temporai_subsample_rate) * 100 : .1f}%"
)

TemporAI uses two dataframes, a static dataframe of shape (249, 1) and a time series dataframe of shape (51954, 13761). This is sub-sampled at a rate of  2.1%


Let's save these dataframes to disk, so we can inspect their disk cost and the memory cost to re-load them from scratch.

In [7]:
save_dir = Path("./speed_comparisons/temporai/compressed")

In [11]:
%%time
%%memit

save_dir.mkdir(parents=True, exist_ok=True)

temporai_static.to_parquet(save_dir / "static.parquet")
temporai_ts.to_parquet(save_dir / "ts.parquet")

uncompressed_save_dir = Path("./speed_comparisons/temporai/uncompressed")
uncompressed_save_dir.mkdir(parents=True, exist_ok=True)

temporai_static.to_parquet(uncompressed_save_dir / "static.parquet", compression=None)
temporai_ts.to_parquet(uncompressed_save_dir / "ts.parquet", compression=None)

compressed_temporai_size = sum(f.stat().st_size for f in save_dir.glob('**/*') if f.is_file())
uncompressed_temporai_size = sum(f.stat().st_size for f in uncompressed_save_dir.glob('**/*') if f.is_file())

print(
    f"The compressed data takes up {naturalsize(compressed_temporai_size)} on disk.\n"
    f"The uncompressed data takes up {naturalsize(uncompressed_temporai_size)} on disk "
    "(this is a good approximation of memory cost as it is uncompressed)."
)

print(
    f"Scaling for the subsampling to only {n_subjects} patients, this would equate to approximately:\n"
    f"The compressed data takes up {naturalsize(compressed_temporai_size/subsample_rate, format='%.3f')} "
    " on disk.\n"
    f"The uncompressed data takes up {naturalsize(uncompressed_temporai_size/subsample_rate, format='%.3f')} "
    "on disk (this is a good approximation of memory cost)."
)

The compressed data takes up 62.3 MB on disk.
The uncompressed data takes up 70.2 MB on disk (this is a good approximation of memory cost as it is uncompressed).
Scaling for the subsampling to only 250 patients, this would equate to approximately: The compressed data takes up 3.0 GB on disk.
The uncompressed data takes up 3.4 GB on disk (this is a good approximation of memory cost as it is uncompressed).
peak memory: 11097.99 MiB, increment: 212.54 MiB
CPU times: user 32.3 s, sys: 1.73 s, total: 34.1 s
Wall time: 36.5 s


In [8]:
%%time
%%memit

temporai_static = pd.read_parquet(save_dir / "static.parquet")
temporai_ts = pd.read_parquet(save_dir / "ts.parquet")

peak memory: 9078.91 MiB, increment: 8683.60 MiB
CPU times: user 7.64 s, sys: 5.55 s, total: 13.2 s
Wall time: 4.82 s


Recall that this is under the effect of sub-sampling, so a true memory cost of loading all the data in this format would have a peak of approximately 8719.1 * (100/2.1) = 415 GB!

TemporAI generally converts their timeseries data into a dense, 3D matrix across samples, timepoints, and features. For use in ML pipelines, this is then generally iterated through directly via simple numpy iteration. 

For example: 
  * Datasets are converted to 3D views here: https://github.com/vanderschaarlab/temporai/blob/main/src/tempor/plugins/prediction/one_off/classification/__init__.py#L59 and https://github.com/vanderschaarlab/temporai/blob/67ebd74dc24728163d9aec37f1771a83fc3346e2/src/tempor/data/utils.py#L49
  * Iteration through numpy arrays happens here: https://github.com/vanderschaarlab/temporai/blob/main/src/tempor/models/ddh.py#L155
  
Though a full comparison warrants use of their library (and will further depend on the exact model used (as each has different strategies for processing data), we can simulate that approach here quickly:

In [9]:
def no_categories(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if pd.api.types.is_categorical_dtype(df[c]):
            df[c] = df[c].cat.codes
    return df

def to_3D_arr(df: pd.DataFrame, max_timesteps: int | None = None) -> np.ndarray:
    df = no_categories(df)
    samples = set(df.index.get_level_values(0))
    num_samples = len(samples)
    num_features = len(df.columns)
    num_timesteps_per_sample = df.groupby(level=0).size()
    max_actual_timesteps = num_timesteps_per_sample.max()
    max_timesteps = max_actual_timesteps if max_timesteps is None else max_timesteps
    array = np.full(shape=(num_samples, max_timesteps, num_features), fill_value=np.NaN)
    for i_sample, idx_sample in enumerate(samples):
        set_vals = df.loc[idx_sample, :, :].to_numpy()[:max_timesteps, :]  # pyright: ignore
        if i_sample == 0:
            array = array.astype(set_vals.dtype)  # Need to cast to the type matching source data.
        array[i_sample, : num_timesteps_per_sample[idx_sample], :] = set_vals  # pyright: ignore
    return array

In [10]:
class SimpleTemporAIStyleDataset(Dataset):
    def __init__(self, static: np.ndarray, ts: np.ndarray):
        self.static = static
        self.ts = ts
        
    def __len__(self) -> int: return self.ts.shape[0]
    
    def __getitem__(self, idx) -> dict[str, torch.Tensor]:
        return {'static': torch.Tensor(self.static[idx]), 'ts': torch.Tensor(self.ts[idx])}
    
def profile_temporai_dataset(
    temporai_static, temporai_ts, batch_size: int = 16,
    n_iter_samples: int = 30,
    max_seq_len: int = 32,
):
    static_as_np = np.nan_to_num(no_categories(temporai_static).to_numpy(), nan=0)
    ts_as_np = np.nan_to_num(to_3D_arr(temporai_ts, max_timesteps=max_seq_len), nan=0)
    print(
        f"Yielded a static NP array of shape {static_as_np.shape} and a TS NP array "
        f"of shape {ts_as_np.shape}."
    )
    temporai_pyd = SimpleTemporAIStyleDataset(static_as_np, ts_as_np)

    profile_batch_iteration_speed_and_cost(
        batch_size=batch_size, pyd=temporai_pyd, n_iter_samples=n_iter_samples
    )

In [ ]:
%%time
%%memit

profile_temporai_dataset(
    temporai_static, temporai_ts, batch_size=16, n_iter_samples=5, max_seq_len=1024
)

Yielded a static NP array of shape (249, 1) and a TS NP array of shape (249, 1024, 13761).


  0%|          | 0/16 [00:00<?, ?it/s]

Sampling Dataloader Iteration Speed:   0%|          | 0/5 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Iterating through an entire dataloader of 16 batches of size 16 took the following time per batch:
0:00:00.596937 ± 0:00:00.020572 (0:00:00.571113-0:00:00.623727)

Total batch size:
877.2 MB ± 95.5 MB (507.3 MB-901.8 MB)
  Size of static:
    62 Bytes ± 6 Bytes (36 Bytes-64 Bytes)
  Size of ts:
    877.2 MB ± 95.5 MB (507.3 MB-901.8 MB)


As we can see, the strategy of featurizing and batching used in TemporAI results (on this synthetic dataset) in a significantly faster iteration speed and a marginally lower memory cost than does the strategy used in ESGPT (all formats are mean ± standard deviation (min - max)

TemporAI Speed: `0:00:00.596937 ± 0:00:00.020572 (0:00:00.571113-0:00:00.623727)`  
ESGPT Speed:    `0:00:00.416088 ± 0:00:00.004160 (0:00:00.410064-0:00:00.422636)`

TemporAI Memory: `877.2 MB ± 95.5 MB (507.3 MB-901.8 MB)`  
ESGPT Memory:    `67.2 MB ± 35.0 MB (18.3 MB-410.6 MB)`

In table form (using chatGPT for conversions, so may need to be double checked), where "Delta" means what % of TemporAI's resource cost does ESGPT _save_ (higher is better), we get the following:
|                      | TemporAI             | ESGPT                | Improvement (%)  |
|----------------------|----------------------|----------------------|-------------------|
| **Speed (ms)**       | 597 ± 20.6 (571 - 624) | 416 ± 4.16 (410 - 423) | 30.3%            |
| **Memory (MB)**      | 877 ± 95.5 (507 - 902) | 67.2 ± 35.0 (18.3 - 411) | 92.3%           |

There are some biases in this format, on both sides:
  1. ESGPT samples different subsequences per item iteration, whereas TemporAI is limited to only using the first max subsequence samples. 
  2. This dataset has relatively few measurements, which will reduce the memory disparity between the two formats (this bias favors TemporAI).
  3. The strategy of flattening this dataset may induce too much memory overhead, as if multiple measurements are not common within an event, it will have extra columns that TemporAI does not need. Conversely, it may reduce a significant amount of data, as if there are many measurements than a simple count, sum, sum_sqd, min, and max representation will not fully capture the data, thereby reducing the burden on TemporAI. (This bias could favor either).
  4. This example uses a long max sequence length, which opens up more opportunities for ESGPT's dynamic padding scheme to save memory. (This bias favors ESGPT on Memory).
  5. This example uses a long max sequence length, which means that ESGPT's per-event measurement padding workflow will take longer per batch (This bias favors TemporAI on speed).
  
### omop-learn

Next, we'll attempt to compare against [omop-Learn](https://github.com/clinicalml/omop-learn/tree/master). omop-learn uses a similar storage model to ESGPT (a _long_ format) but it does not include the storage of numerical values, only codes, and it similarly does not allow for randomly sampled sub-sequences per dataset item. See [here](https://github.com/clinicalml/omop-learn/blob/a33440af2b9f1342e0c16106acf93131b9369441/src/omop_learn/torch/data.py) for more details. To simulate this, we'll first convert the ESGPT data into a pre-tokenized OMOP-learn representation, store in the desired format (JSON), then re-load it and iterate through it in the same manner as omop-learn does. As static data are not separately encoded in the omop-learn dataset format, we'll omit that as well here.

In [7]:
from typing import Any
import json, shutil, pandas as pd, polars as pl
REFTIME = pd.Timestamp("2000-01-01")

def to_unixtime(str_time_array):
    unix_times = (pd.to_datetime(str_time_array) - REFTIME) // pd.Timedelta("1d")
    return unix_times


def from_unixtime(unix_time_array):
    datetimes = [pd.to_datetime(t * pd.Timedelta("1d") + REFTIME) for t in unix_time_array]
    return datetimes

def ESD_row_to_omoplearn_row(row: dict[str, Any], unified_vocab: dict[int, str]) -> dict[str, Any]:
    example = {}
    
    # Dates
    example['dates'] = [str(row['start_time'] + timedelta(minutes=t)) for t in row['time']]
    
    # Visits
    example['visits'] = [
        [unified_vocab[idx] for idx in indices] for indices in row['dynamic_indices']
    ]
    
    example['tok_visits'] = row['dynamic_indices']
    
    assert len(example['dates']) == len(example['visits'])
    assert len(example['dates']) == len(example['tok_visits'])
    
    example['y'] = 0
    
    return example

def ESD_to_omoplearn(ESD: Dataset, n_subjects: int | None) -> list[dict[str, Any]]:
    if n_subjects is None:
        subject_ids = list(ESD.split_subjects['train'])
    else:
        subject_ids = list(np.random.default_rng(1).choice(list(ESD.split_subjects['train']), size=n_subjects))
    
    unified_vocab = {}
    for m, idxmap in ESD.unified_vocabulary_idxmap.items():
        for k, v in idxmap.items():
            unified_vocab[v] = f"{m}/{k}"
    
    omop_file_dir = Path("./omop_reps")
    if omop_file_dir.is_dir():
        shutil.rmtree(omop_file_dir)

    omop_file_dir.mkdir(exist_ok=True, parents=True)
    sp = "train"
    
    omop_sp_dir = omop_file_dir / sp
    omop_sp_dir.mkdir(exist_ok=True, parents=True)
    for fp in tqdm(
        list((ESD.config.save_dir / "DL_reps").glob(f"{sp}_*.parquet")), desc="File", leave=False
    ):
        DL_reps_df = (
            pl.scan_parquet(fp)
            .filter(pl.col('subject_id').is_in(subject_ids))
            .select('subject_id', 'start_time', 'time', 'dynamic_indices')
            .collect()
        )
        cols = DL_reps_df.columns
        rows = DL_reps_df.rows()

        for row in rows:
            row_as_dict = {col: val for col, val in zip(cols, row)}
            example = ESD_row_to_omoplearn_row(row_as_dict, unified_vocab)
            with open(omop_sp_dir / "data.json", "a") as f:
                json.dump(example, f)
                f.write('\n')

In [9]:
%%time
%%memit
n_subjects = 250
omop_subsample_rate = n_subjects / len(ESD.split_subjects['train'])
ESD_to_omoplearn(ESD, n_subjects=n_subjects)

File:   0%|          | 0/4 [00:00<?, ?it/s]

peak memory: 631.82 MiB, increment: 228.32 MiB
CPU times: user 10.4 s, sys: 673 ms, total: 11.1 s
Wall time: 19 s


In [10]:
JSON_dataset_size = sum(f.stat().st_size for f in Path("./omop_reps").glob('**/*') if f.is_file())
print(
    f"The JSON dumped dataset takes up {naturalsize(JSON_dataset_size)} on disk.\n"
    "Given subsampling, this equates to approximately "
    f"{naturalsize(JSON_dataset_size/omop_subsample_rate, format='%.3f')}."
)

The JSON dumped dataset takes up 69.6 MB on disk.
Given subsampling, this equates to approximately 2.700 GB.


In [8]:
# Lightly adapted from https://github.com/clinicalml/omop-learn/blob/a33440af2b9f1342e0c16106acf93131b9369441/src/omop_learn/torch/data.py

from collections import Counter

class OMOPDatasetTorch(torch.utils.data.Dataset):
    def __init__(
        self,
        omop_dataset_file,
        max_num_visits=None,
    ):
        super().__init__()
        self.items = {}
        self.visit_sequences = []  # patient x (# visits for patient) lists w/ concepts expressed
        self.time_sequences = []  # patient x (# visits for patient) times of visits
        self.visit_sizes = []  # patient x (# visits for patient) # concepts in each visit
        self.outcomes = []  # patient--outcome for each patient
        self.tok_visit_sequences = None
        self.tokenizer = None
        self.max_num_visits = max_num_visits  # if set, truncate to most recent
        self._load_json(omop_dataset_file, False)


    def _load_json(self, path_to_json, tokenize_on_load):
        # read once to build concept set
        # (and load visits if tokenize_on_load=False)
        concept_set = set()
        concept_counts = Counter()
        concept_counts_by_year = Counter()
        years = set()
        max_num_visits = 0
        skipped = 0
        with open(path_to_json) as json_fh:
            for i, line in enumerate(json_fh.readlines()):
                example = self._process_line(line)
                max_num_visits = max(max_num_visits, len(example['visits']))

                for time, visit in zip(example['unix_times'], example['visits']):
                    for concept in visit:
                        concept_set.add(concept)

                if len(example['visits']) == 0:
                    skipped += 1
                    continue

                if i == 0:
                    for key, value in example.items():
                        self.items[key] = [value]
                else:
                    # correctly gives error when key is not found
                    # already; all items need to have exactly the same
                    # set of keys.
                    for key,value in example.items():
                        self.items[key].append(value)

        print(f"Skipped {skipped} patients for empty visit lists")
        if not self.max_num_visits:
            self.max_num_visits = max_num_visits

#         if not self.tokenizer:
#             self.tokenizer = ConceptTokenizer(concept_set)
#             print("built tokenizer")

#         # read again to build tokenized visits
#         if tokenize_on_load:
#             self.items['tok_visits'] = []
#             with open(path_to_json, "r") as json_fh:
#                 for i, line in enumerate(json_fh.readlines()):
#                     example = self._process_line(line)
#                     tok_visit_list = []
#                     for visit in example['visits']:
#                         tok_visit = self.tokenizer.concepts_to_ids(visit)
#                         tok_visit_list.append(tok_visit)
#                     if len(tok_visit_list) > 0:
#                         self.items['tok_visits'].append(tok_visit_list)

        self.outcomes = torch.LongTensor(self.items['y'])
        self.one_fraction = self.outcomes.sum() / len(self.outcomes)
        self.one_odds = self.one_fraction / (1 - self.one_fraction)

    def _process_line(self, line):
        example = json.loads(line)
        dates = example['dates']
        unix_times = to_unixtime(dates)
        example['unix_times'] = unix_times

        # make sure visits are sorted by date
        # This actually contains a minor error correction in omop-learn; namely, that pre-tokenized visits may not
        # have been properly sorted.
        sorted_visits = [v for d,v in sorted(zip(example['unix_times'], example['visits']))]
        if 'tok_visits' in example:
            sorted_tok_visits = [t for d,t in sorted(zip(example['unix_times'], example['tok_visits']))]
            example['tok_visits'] = sorted_tok_visits
        example['visits'] = sorted_visits
        example['unix_times'] = sorted(example['unix_times'])
        example['dates'] = sorted(example['dates'])
        
        assert len(example['visits']) == len(example['unix_times'])
        assert len(example['tok_visits']) == len(example['unix_times'])

        return example

    def __getitem__(self, idx):
        example = {k : v[idx] for k,v in self.items.items()}
        times = torch.LongTensor(example['unix_times'])
        visits = example['visits'] if 'tok_visits' not in example else example['tok_visits']
        assert len(times) == len(visits)

        # trim before tokenizing
        visits = visits[-self.max_num_visits :]
        times = times[-self.max_num_visits :]

        if 'tok_visits' not in example:
            raise NotImplementedError(f"Must be pre-tokenized!")
#             tok_visits = []
#             for visit in example['visits']:
#                 tok_visits.append(self.tokenizer.concepts_to_ids(visit))
#             visits = tok_visits

        example['visits'] = visits
        # Another small bug correction
        example['unix_times'] = times

        visit_sizes = torch.LongTensor([len(v) for v in visits])
        outcome = self.outcomes[idx]
        nvisits = len(visits)

        return example

    # pads a batch to largest # of visits / patient in the batch
    # and largest # of concepts / visit along concept dim.
    def collate(self, batch):
        # first group along dict keys
        batch_collated = {}
        for k in batch[0].keys():
            batch_collated[k] = [b[k] for b in batch]

        keys = list(batch_collated.keys())
        N = len(batch_collated['y'])

        # each patient is a list of visits
        max_num_visits = max([len(v) for v in batch_collated['visits']])
        max_num_concepts = max(l for p in range(N) for l in [len(v) for v in batch_collated['visits'][p]])

        concept_tensor = torch.full(
            (N, max_num_visits, max_num_concepts),
            0,
            dtype=torch.long,
        )

        times_tensor = torch.full((N, max_num_visits), -1, dtype=torch.long)
        batch = batch_collated

        lengths = torch.zeros(N)
        for i, visit_list in enumerate(batch['visits']):
            assert len(visit_list) == len(batch['unix_times'][i]), (
                f"Visits don't match! Got {len(visit_list)} visits and {len(batch['unix_times'][i])} times "
                f"for batch element {i}"
            )
            num_visits = len(visit_list)  # visits of this patient we are including
            lengths[i] = num_visits
            for j, visit in enumerate(visit_list):
                visit_size = len(batch['visits'][i][j])
                assert(visit_size == len(visit))
                concept_tensor[i, j, : visit_size] = torch.Tensor(visit)
            times_tensor[i, :num_visits] = torch.Tensor(batch['unix_times'][i])
        batch.pop("unix_times")
        batch.pop("dates")
        # Another small correction:
        batch.pop("tok_visits")
        batch["visits"] = concept_tensor
        batch["times"] = times_tensor
        batch["lengths"] = lengths
        for k,v in batch.items():
            if not isinstance(v, torch.Tensor):
                batch[k] = torch.tensor(v)
        return batch

    def __len__(self):
        return len(self.outcomes)

In [12]:
%%time
%%memit
ODT = OMOPDatasetTorch("./omop_reps/train/data.json", max_num_visits=1024)

Skipped 0 patients for empty visit lists
peak memory: 830.48 MiB, increment: 314.32 MiB
CPU times: user 1.47 s, sys: 318 ms, total: 1.78 s
Wall time: 3.1 s


In [24]:
%%time
%%memit

profile_batch_iteration_speed_and_cost(
    batch_size=16, pyd=ODT, n_iter_samples=5, collate_fn=ODT.collate
)

  0%|          | 0/16 [00:00<?, ?it/s]

Sampling Dataloader Iteration Speed:   0%|          | 0/5 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Iterating through an entire dataloader of 16 batches of size 16 took the following time per batch:
0:00:00.174998 ± 0:00:00.016615 (0:00:00.142167-0:00:00.186295)

Total batch size:
25.3 MB ± 10.7 MB (9.5 MB-46.1 MB)
  Size of visits:
    25.3 MB ± 10.7 MB (9.5 MB-46.0 MB)
  Size of y:
    123 Bytes ± 17 Bytes (56 Bytes-128 Bytes)
  Size of times:
    83.0 kB ± 34.8 kB (33.5 kB-131.1 kB)
  Size of lengths:
    61 Bytes ± 8 Bytes (28 Bytes-64 Bytes)
peak memory: 1103.59 MiB, increment: 184.39 MiB
CPU times: user 36 s, sys: 5.75 s, total: 41.7 s
Wall time: 18 s


This, as we can see, is faster and smaller than either of the other two alternates. However, recall that this is not a fair comparison; this system ignores continuous values, event masks, and static variables. We can try to assess the impact of these by altering the omop-learn format to include such data, which we'll explore below:

### Altered omop-learn

In [9]:
def ESD_row_to_altered_omoplearn_row(row: dict[str, Any], unified_vocab: dict[int, str]) -> dict[str, Any]:
    example = {}
    
    # Dates
    example['dates'] = [str(row['start_time'] + timedelta(minutes=t)) for t in row['time']]
    
    # Visits
    example['visits'] = [
        [unified_vocab[idx] for idx in indices] for indices in row['dynamic_indices']
    ]
    
    example['tok_visits'] = row['dynamic_indices']
    
    assert len(example['dates']) == len(example['visits'])
    assert len(example['dates']) == len(example['tok_visits'])
    
    example['visit_nums'] = row['dynamic_values']
    assert len(example['dates']) == len(example['visit_nums'])
    for v, vn in zip(example['visits'], example['visit_nums']):
        assert len(v) == len(vn), f"Misaligned in conversion! {len(v)} v. {len(vn)}."

    example['static'] = row['static_indices']
    
    example['y'] = 0
    
    return example

def ESD_to_altered_omoplearn(ESD: Dataset, n_subjects: int | None) -> list[dict[str, Any]]:
    if n_subjects is None:
        subject_ids = list(ESD.split_subjects['train'])
    else:
        subject_ids = list(np.random.default_rng(1).choice(list(ESD.split_subjects['train']), size=n_subjects))
    
    unified_vocab = {}
    for m, idxmap in ESD.unified_vocabulary_idxmap.items():
        for k, v in idxmap.items():
            unified_vocab[v] = f"{m}/{k}"
    
    omop_file_dir = Path("./altered_omop_reps")
    if omop_file_dir.is_dir():
        shutil.rmtree(omop_file_dir)

    omop_file_dir.mkdir(exist_ok=True, parents=True)
    sp = "train"
    
    omop_sp_dir = omop_file_dir / sp
    omop_sp_dir.mkdir(exist_ok=True, parents=True)
    for fp in tqdm(
        list((ESD.config.save_dir / "DL_reps").glob(f"{sp}_*.parquet")), desc="File", leave=False
    ):
        DL_reps_df = (
            pl.scan_parquet(fp)
            .filter(pl.col('subject_id').is_in(subject_ids))
            .select('subject_id', 'start_time', 'time', 'dynamic_indices', 'dynamic_values', 'static_indices')
            .collect()
        )
        cols = DL_reps_df.columns
        rows = DL_reps_df.rows()

        for row in rows:
            row_as_dict = {col: val for col, val in zip(cols, row)}
            example = ESD_row_to_altered_omoplearn_row(row_as_dict, unified_vocab)
            with open(omop_sp_dir / "data.json", "a") as f:
                for v, vn in zip(example['visits'], example['visit_nums']):
                    assert len(v) == len(vn), f"Misaligned in conversion! {len(v)} v. {len(vn)}."
                json_str = json.dumps(example)
                f.write(f"{json_str}\n")
                reloaded = json.loads(json_str)
                for i, (v, vn) in enumerate(zip(reloaded['visits'], reloaded['visit_nums'])):
                    if len(v) != len(vn):
                        print(example['visit_nums'][i])
                        print(example['visits'][i])
                        raise ValueError(f"Misaligned in conversion! {len(v)} v. {len(vn)}.")


In [35]:
%%time
%%memit
n_subjects = 250
omop_subsample_rate = n_subjects / len(ESD.split_subjects['train'])
ESD_to_altered_omoplearn(ESD, n_subjects=n_subjects)

File:   0%|          | 0/4 [00:00<?, ?it/s]

peak memory: 1383.14 MiB, increment: 415.16 MiB
CPU times: user 20.2 s, sys: 1.89 s, total: 22.1 s
Wall time: 23.1 s


In [15]:
omop_subsample_rate = 250 / len(ESD.split_subjects['train'])
print(omop_subsample_rate)

0.025767882910740055


In [36]:
JSON_dataset_size = sum(f.stat().st_size for f in Path("./altered_omop_reps").glob('**/*') if f.is_file())
print(
    f"The JSON dumped dataset takes up {naturalsize(JSON_dataset_size)} on disk.\n"
    "Given subsampling, this equates to approximately "
    f"{naturalsize(JSON_dataset_size/omop_subsample_rate, format='%.3f')}."
)

The JSON dumped dataset takes up 107.6 MB on disk.
Given subsampling, this equates to approximately 4.175 GB.


In [10]:
class AlteredOMOPDatasetTorch(torch.utils.data.Dataset):
    def __init__(
        self,
        omop_dataset_file,
        max_num_visits=None,
    ):
        super().__init__()
        self.items = {}
        self.visit_sequences = []  # patient x (# visits for patient) lists w/ concepts expressed
        self.visit_nums_sequences = []
        self.static_sequences = []
        self.time_sequences = []  # patient x (# visits for patient) times of visits
        self.visit_sizes = []  # patient x (# visits for patient) # concepts in each visit
        self.outcomes = []  # patient--outcome for each patient
        self.tok_visit_sequences = None
        self.tokenizer = None
        self.max_num_visits = max_num_visits  # if set, truncate to most recent
        self._load_json(omop_dataset_file, False)


    def _load_json(self, path_to_json, tokenize_on_load):
        # read once to build concept set
        # (and load visits if tokenize_on_load=False)
        concept_set = set()
        concept_counts = Counter()
        concept_counts_by_year = Counter()
        years = set()
        max_num_visits = 0
        skipped = 0
        with open(path_to_json) as json_fh:
            for i, line in enumerate(json_fh.readlines()):
                example = self._process_line(line)
                max_num_visits = max(max_num_visits, len(example['visits']))

                for time, visit in zip(example['unix_times'], example['visits']):
                    for concept in visit:
                        concept_set.add(concept)

                if len(example['visits']) == 0:
                    skipped += 1
                    continue

                if i == 0:
                    for key, value in example.items():
                        self.items[key] = [value]
                else:
                    # correctly gives error when key is not found
                    # already; all items need to have exactly the same
                    # set of keys.
                    for key,value in example.items():
                        self.items[key].append(value)

        print(f"Skipped {skipped} patients for empty visit lists")
        if not self.max_num_visits:
            self.max_num_visits = max_num_visits

#         if not self.tokenizer:
#             self.tokenizer = ConceptTokenizer(concept_set)
#             print("built tokenizer")

#         # read again to build tokenized visits
#         if tokenize_on_load:
#             self.items['tok_visits'] = []
#             with open(path_to_json, "r") as json_fh:
#                 for i, line in enumerate(json_fh.readlines()):
#                     example = self._process_line(line)
#                     tok_visit_list = []
#                     for visit in example['visits']:
#                         tok_visit = self.tokenizer.concepts_to_ids(visit)
#                         tok_visit_list.append(tok_visit)
#                     if len(tok_visit_list) > 0:
#                         self.items['tok_visits'].append(tok_visit_list)

        self.outcomes = torch.LongTensor(self.items['y'])
        self.one_fraction = self.outcomes.sum() / len(self.outcomes)
        self.one_odds = self.one_fraction / (1 - self.one_fraction)

    def _process_line(self, line):
        example = json.loads(line)
        dates = example['dates']
        unix_times = to_unixtime(dates)
        example['unix_times'] = unix_times

        # make sure visits are sorted by date
        # This actually contains a minor error correction in omop-learn; namely, that pre-tokenized visits may not
        # have been properly sorted.
#         sorted_visits = [v for d,v in sorted(zip(example['unix_times'], example['visits']))]
#         example['visits'] = sorted_visits
#         if 'tok_visits' in example:
#             sorted_tok_visits = [t for d,t in sorted(zip(example['unix_times'], example['tok_visits']))]
#             example['tok_visits'] = sorted_tok_visits
#         if 'visit_nums' in example:
#             sorted_visit_nums = [t for d,t in sorted(zip(example['unix_times'], example['visit_nums']))]
#             example['visit_nums'] = sorted_visit_nums
#             assert len(example['visit_nums']) == len(example['visits'])
#             for v, vn in zip(example['visits'], example['visit_nums']):
#                 assert len(v) == len(vn), f"Error in parsing! {len(v)} vs. {len(vn)}."
            
#         example['unix_times'] = sorted(example['unix_times'])
#         example['dates'] = sorted(example['dates'])
        
#         assert len(example['visits']) == len(example['unix_times'])
#         assert len(example['tok_visits']) == len(example['unix_times'])

        return example

    def __getitem__(self, idx):
        example = {k : v[idx] for k,v in self.items.items()}
        times = torch.LongTensor(example['unix_times'])
        visits = example['visits'] if 'tok_visits' not in example else example['tok_visits']
        assert len(times) == len(visits)
        assert len(times) == len(example['visit_nums'])
        
        for v, vn in zip(visits, example['visit_nums']):
            assert len(v) == len(vn), f"Error! {len(v)} vs. {len(vn)}."

        # trim before tokenizing
        visits = visits[-self.max_num_visits :]
        times = times[-self.max_num_visits :]
        example['visit_nums'] = example['visit_nums'][-self.max_num_visits:]

        if 'tok_visits' not in example:
            raise NotImplementedError(f"Must be pre-tokenized!")
#             tok_visits = []
#             for visit in example['visits']:
#                 tok_visits.append(self.tokenizer.concepts_to_ids(visit))
#             visits = tok_visits

        example['visits'] = visits
        # Another small bug correction
        example['unix_times'] = times

        visit_sizes = torch.LongTensor([len(v) for v in visits])
        outcome = self.outcomes[idx]
        nvisits = len(visits)

        return example

    # pads a batch to largest # of visits / patient in the batch
    # and largest # of concepts / visit along concept dim.
    def collate(self, batch):
        # first group along dict keys
        batch_collated = {}
        for k in batch[0].keys():
            batch_collated[k] = [b[k] for b in batch]

        keys = list(batch_collated.keys())
        N = len(batch_collated['y'])

        # each patient is a list of visits
        max_num_visits = max([len(v) for v in batch_collated['visits']])
        max_num_concepts = max(l for p in range(N) for l in [len(v) for v in batch_collated['visits'][p]])

        concept_tensor = torch.full(
            (N, max_num_visits, max_num_concepts),
            0,
            dtype=torch.long,
        )
        
        nums_tensor = torch.full(
            (N, max_num_visits, max_num_concepts),
            0,
            dtype=torch.float32,
        )
        nums_mask_tensor = torch.full(
            (N, max_num_visits, max_num_concepts),
            0,
            dtype=torch.bool,
        )

        times_tensor = torch.full((N, max_num_visits), -1, dtype=torch.long)
        batch = batch_collated

        lengths = torch.zeros(N)
        for i, visit_list in enumerate(batch['visits']):
            assert len(visit_list) == len(batch['unix_times'][i]), (
                f"Visits don't match! Got {len(visit_list)} visits and {len(batch['unix_times'][i])} times "
                f"for batch element {i}"
            )
            assert len(visit_list) == len(batch['visit_nums'][i]), (
                f"Visits don't match! Got {len(visit_list)} visits and {len(batch['visit_nums'][i])} nums "
                f"for batch element {i}"
            )
            num_visits = len(visit_list)  # visits of this patient we are including
            lengths[i] = num_visits
            for j, visit in enumerate(visit_list):
                visit_size = len(batch['visits'][i][j])
                visit_nums = batch['visit_nums'][i][j]
                
                assert(visit_size == len(visit))
                assert visit_size == len(visit_nums), f"Sizes differ! {visit_size} vs {len(visit_nums)}"
                
                concept_tensor[i, j, : visit_size] = torch.Tensor(visit)
                nums_tensor[i, j, : visit_size] = torch.Tensor(
                    [0 if x is None else x for x in visit_nums]
                )
                nums_mask_tensor[i, j, : visit_size] = torch.Tensor(
                    [x is not None for x in visit_nums]
                )
            times_tensor[i, :num_visits] = torch.Tensor(batch['unix_times'][i])
            
        batch.pop("unix_times")
        batch.pop("dates")
        # Another small correction:
        batch.pop("tok_visits")
        batch.pop("visit_nums")
        batch["visits"] = concept_tensor
        batch["visits_values"] = nums_tensor
        batch["visits_values_mask"] = nums_mask_tensor
        batch["times"] = times_tensor
        batch["lengths"] = lengths
        for k,v in batch.items():
            if not isinstance(v, torch.Tensor):
                batch[k] = torch.tensor(v)
        return batch

    def __len__(self):
        return len(self.outcomes)

In [11]:
%%time
%%memit
AlteredODT = AlteredOMOPDatasetTorch("./altered_omop_reps/train/data.json", max_num_visits=1024)

Skipped 0 patients for empty visit lists
peak memory: 839.12 MiB, increment: 442.97 MiB
CPU times: user 2.24 s, sys: 303 ms, total: 2.54 s
Wall time: 3.45 s


In [44]:
%%time
%%memit

profile_batch_iteration_speed_and_cost(
    batch_size=16, pyd=AlteredODT, n_iter_samples=5, collate_fn=AlteredODT.collate
)

  0%|          | 0/16 [00:00<?, ?it/s]

Sampling Dataloader Iteration Speed:   0%|          | 0/5 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Sampling Batch:   0%|          | 0/16 [00:00<?, ?it/s]

Iterating through an entire dataloader of 16 batches of size 16 took the following time per batch:
0:00:00.439344 ± 0:00:00.038233 (0:00:00.394279-0:00:00.497850)

Total batch size:
40.1 MB ± 19.7 MB (7.3 MB-88.7 MB)
  Size of visits:
    24.6 MB ± 12.1 MB (4.5 MB-54.5 MB)
  Size of static:
    123 Bytes ± 17 Bytes (56 Bytes-128 Bytes)
  Size of y:
    123 Bytes ± 17 Bytes (56 Bytes-128 Bytes)
  Size of visits_values:
    12.3 MB ± 6.1 MB (2.2 MB-27.3 MB)
  Size of visits_values_mask:
    3.1 MB ± 1.5 MB (561.8 kB-6.8 MB)
  Size of times:
    80.9 kB ± 36.2 kB (25.5 kB-131.1 kB)
  Size of lengths:
    61 Bytes ± 8 Bytes (28 Bytes-64 Bytes)
peak memory: 1786.18 MiB, increment: 145.68 MiB
CPU times: user 1min, sys: 6.94 s, total: 1min 7s
Wall time: 45 s
